# GBasis Tutorial - Basis Sets

# 1 Basis Sets

In [gbasis](http://gbasis.qcdevs.org/intro.html), a basis is defined as a list of [`GeneralizedContractionShell`](http://gbasis.qcdevs.org/_autosummary/gbasis.html#module-gbasis.contractions) objects, each one representing a linear combination of primitives and containing all the information defining this combination (i.e the angular momentum of the shell, the center of the shell, the exponents of the primitives, the contraction coefficients, and the normalization coefficients).

This notebook showcases the different ways to define a basis set for a molecule and how to transform them.

## 1.1 Loading Basis Sets

Basis set information is usually available in text format, such as from the [Basis Set Exchange](https://www.basissetexchange.org/). In [gbasis](http://gbasis.qcdevs.org/intro.html), two of the most popular formats are supported:
  - Gaussian94 (.gbs)
  - NWChem (.nw).

The following example shows how to load a basis set from a Gaussian94  or a NWChem file.

### 1.1.1 Gaussian94 format

In [37]:
import numpy as np
from gbasis.parsers import parse_gbs, parse_nwchem

# load hydrogen atom def2-SVP basis set information with the Gaussian94 format
gbs_basis_dict = parse_gbs("hydrogen_def2-svp.1.gbs")

# basis set information is stored as
# {'Atom Symbol': [(Angular Momentum, [Exponents], [Coefficients]), ...]}
print("def2-SVP Basis Set Loaded from Gaussian94 Format:")
for atom in gbs_basis_dict:
    print(f"Atom: {atom}")
    print(f"   Number of shells: {len(gbs_basis_dict[atom])}")
    for shell_num, shell in enumerate(gbs_basis_dict[atom]):
        print(f"   Shell {shell_num} has angular momentum {shell[0]}")
        print(f"   Shell {shell_num} has exponents {shell[1]}")
        print(f"   Shell {shell_num} has coefficients {shell[2].flatten()}")


def2-SVP Basis Set Loaded from Gaussian94 Format:
Atom: H
   Number of shells: 3
   Shell 0 has angular momentum 0
   Shell 0 has exponents [13.010701    1.9622572   0.44453796]
   Shell 0 has coefficients [0.01968216 0.13796524 0.47831935]
   Shell 1 has angular momentum 0
   Shell 1 has exponents [0.12194962]
   Shell 1 has coefficients [1.]
   Shell 2 has angular momentum 1
   Shell 2 has exponents [0.8]
   Shell 2 has coefficients [1.]


### 1.1.2 NwChem format

In [38]:
print("Loading def2-SVP Basis Set from NWChem Format...")
nw_basis_dict = parse_nwchem("hydrogen_def2-svp.1.nw")

print("Checking that the Gaussian94 and NWChem loaded basis sets are the same:")
print("Both loaded basis sets have the same:")
for atom in gbs_basis_dict:
    print(f"Atom: {atom} is the same on both cases: {atom in nw_basis_dict}")
    print(f"Number of shells is the same: {len(gbs_basis_dict[atom]) == len(nw_basis_dict[atom])}")
    # check that the angular momentum, exponents, and coefficients are the same for each shell
    contractions_pair = enumerate(zip(gbs_basis_dict[atom], nw_basis_dict[atom]))
    for shell_num, (gb_shell, nw_shell) in contractions_pair:
        print(
            f"Shell {shell_num} has the same angular momentum: {gb_shell[0] == nw_shell[0]}"
        )
        print(
            f"Shell {shell_num} has the same exponents: {np.allclose(gb_shell[1], nw_shell[1])}"
        )
        print(
            f"Shell {shell_num} has the same coefficients: {np.allclose(gb_shell[2], nw_shell[2])}"
        )

Loading def2-SVP Basis Set from NWChem Format...
Checking that the Gaussian94 and NWChem loaded basis sets are the same:
Both loaded basis sets have the same:
Atom: H is the same on both cases: True
Number of shells is the same: True
Shell 0 has the same angular momentum: True
Shell 0 has the same exponents: True
Shell 0 has the same coefficients: True
Shell 1 has the same angular momentum: True
Shell 1 has the same exponents: True
Shell 1 has the same coefficients: True
Shell 2 has the same angular momentum: True
Shell 2 has the same exponents: True
Shell 2 has the same coefficients: True


## 1.2 Building Basis Sets

Once the data for a basis set for a list of atoms is loaded, gbasis allows to build the contraction shells for a molecule. This is done by the `make_contraction_shells` function of the [`parsers`](http://gbasis.qcdevs.org/_autosummary/gbasis.html#module-gbasis.parsers) module. The following example shows how to use this function to build the contraction shells for $\mathrm{H}_{2}$.


In [39]:
from gbasis.parsers import make_contractions

# use H2 molecule as an example system
atoms = ["H", "H"]
atcoords = np.array([[0, 0, 0], [0, 0, 1]])

# make contractions for the hydrogen def2-SVP basis set
gbs_basis = make_contractions(gbs_basis_dict, atoms, atcoords)

print("Number of contracted basis functions:", len(gbs_basis))  # 3 for each H atom
print("Showing first three contraction shells:")
for i, basis in enumerate(gbs_basis[:3]):
    print(f"Contraction shell #{i}")
    print(f"   Center: {basis.coord}")
    print(f"   Angular momentum: {basis.angmom}")
    print(f"   Primitive coefficients {basis.coeffs.T}")
    print(f"   Primitive exponents {basis.exps}")
    print(f"   Primitive normalization constant {basis.norm_cont}")

Number of contracted basis functions: 6
Showing first three contraction shells:
Contraction shell #0
   Center: [0. 0. 0.]
   Angular momentum: 0
   Primitive coefficients [[0.01968216 0.13796524 0.47831935]]
   Primitive exponents [13.010701    1.9622572   0.44453796]
   Primitive normalization constant [[1.70131166]]
Contraction shell #1
   Center: [0. 0. 0.]
   Angular momentum: 0
   Primitive coefficients [[1.]]
   Primitive exponents [0.12194962]
   Primitive normalization constant [[1.]]
Contraction shell #2
   Center: [0. 0. 0.]
   Angular momentum: 1
   Primitive coefficients [[1.]]
   Primitive exponents [0.8]
   Primitive normalization constant [[1. 1. 1.]]


## 1.3 Interfacing with other packages

[gbasis](http://gbasis.qcdevs.org/intro.html) has been designed to be easily interfaced the [iodata](https://iodata.readthedocs.io/en/latest/) package which handles the inputs and outputs for different quantum chemistry formats, such as Gaussian formatted checkpoint files (.fchk) and AIM wavefunction files (.wfn and .wfx). Additionally, [gbasis](http://gbasis.qcdevs.org/intro.html) can be directly interfaced with [pySCF](https://pyscf.org/), a popular Python package for quantum chemistry calculations.

### 1.3.1 [iodata](https://iodata.readthedocs.io/en/latest/) package
This is the preferred way to load basis information from computational chemistry calculations in [gbasis](http://gbasis.qcdevs.org/intro.html). The following example shows how to load a basis set from a Gaussian formatted checkpoint using the [`iodata`](https://iodata.readthedocs.io/en/latest/) package. The same approach can be used for loading the basis information of other formats supported by [`iodata`](https://iodata.readthedocs.io/en/latest/).

In [40]:
from iodata import load_one
from gbasis.wrappers import from_iodata

# load a basis from an fchk file (water at uwB97XD/def2-TZVPD LOT) using IOData
iodata_mol = load_one("water.fchk")
basis, coord_types = from_iodata(iodata_mol)


# print the basis set information
print("Basis loaded from IOData:")
print("Number of contracted basis functions:", len(basis))
print("Showing first three contraction shells:")
for i, basis in enumerate(gbs_basis[:3]):
    print(f"Contraction shell #{i}")
    print(f"   Center: {basis.coord}")
    print(f"   Angular momentum: {basis.angmom}")
    print(f"   Primitive coefficients {basis.coeffs.T}")
    print(f"   Primitive exponents {basis.exps}")
    print(f"   Primitive normalization constant {basis.norm_cont}")

Basis loaded from IOData:
Number of contracted basis functions: 24
Showing first three contraction shells:
Contraction shell #0
   Center: [0. 0. 0.]
   Angular momentum: 0
   Primitive coefficients [[0.01968216 0.13796524 0.47831935]]
   Primitive exponents [13.010701    1.9622572   0.44453796]
   Primitive normalization constant [[1.70131166]]
Contraction shell #1
   Center: [0. 0. 0.]
   Angular momentum: 0
   Primitive coefficients [[1.]]
   Primitive exponents [0.12194962]
   Primitive normalization constant [[1.]]
Contraction shell #2
   Center: [0. 0. 0.]
   Angular momentum: 1
   Primitive coefficients [[1.]]
   Primitive exponents [0.8]
   Primitive normalization constant [[1. 1. 1.]]


### 1.3.2 [pySCF](https://pyscf.org/) package
Additionally [gbasis](http://gbasis.qcdevs.org/intro.html) has a direct interface with [pySCF](https://pyscf.org/) objects. To showcase this feature a  [pySCF](https://pyscf.org/) molecule object is created and its basis are loaded using [gbasis](http://gbasis.qcdevs.org/intro.html).

In [41]:
from pyscf import gto
from gbasis.wrappers import from_pyscf

molecule = '''O 0 0 0;
H 0 1 0;
H 0 0 1'''

basis_name = "sto-3g"

print("Constructing pySCF molecule:")
print("Atomic coordinates:")
print(molecule)
print(f"Basis set used: {basis_name}")

# build an STO-3G basis for water using PySCF
pyscf_mol = gto.Mole()
pyscf_mol.build(
    atom = molecule,
    basis = basis_name,
)

print("Importing basis set from PySCF to gbasis:")
pyscf_basis = from_pyscf(pyscf_mol)

print("Number of contracted basis functions:", len(pyscf_basis))
print("Showing first three contraction shells:")
for i, basis in enumerate(pyscf_basis[:3]):
    print(f"Contraction shell #{i}")
    print(f"   Center: {basis.coord}")
    print(f"   Angular momentum: {basis.angmom}")
    print(f"   Primitive coefficients {basis.coeffs.T}")
    print(f"   Primitive exponents {basis.exps}")
    print(f"   Primitive normalization constant {basis.norm_cont}")

Constricting pySCF molecule:
Atomic coordinates:
O 0 0 0;
H 0 1 0;
H 0 0 1
Basis set used: sto-3g
Importing basis set from PySCF to gbasis:
Number of contracted basis functions: 5
Showing first three contraction shells:
Contraction shell #0
   Center: [0. 0. 0.]
   Angular momentum: 0
   Primitive coefficients [[0.15432897 0.53532814 0.44463454]]
   Primitive exponents [130.70932    23.808861    6.4436083]
   Primitive normalization constant [[0.99999999]]
Contraction shell #1
   Center: [0. 0. 0.]
   Angular momentum: 0
   Primitive coefficients [[-0.09996723  0.39951283  0.70011547]]
   Primitive exponents [5.0331513 1.1695961 0.380389 ]
   Primitive normalization constant [[0.99999999]]
Contraction shell #2
   Center: [0. 0. 0.]
   Angular momentum: 1
   Primitive coefficients [[0.15591627 0.60768372 0.39195739]]
   Primitive exponents [5.0331513 1.1695961 0.380389 ]
   Primitive normalization constant [[0.99999999 0.99999999 0.99999999]]


## 1.3 Types of Coordinate Systems Used by Basis Functions

In `gbasis`, the user can provide the coordinate system used by each shell of generalized contractions that gets stored in the basis list. The `make_contractions` function is useful for instantiating a basis for a system of atoms based on the `basis_dict` output of the parsers from `gbasis.parsers`. For a given call of `make_contractions`, one can pass either a string ("cartesian" or "spherical") or a list of strings to specify `coord_types`. If a string is passed, all contractions for all atoms are treated as having the same `coord_type`. 
If different shells correspond to different coordinate systems, then a list of the same length as the basis must be provided with each entry being "spherical" or "cartesian" to specify the coordinate system
of the corresponding shell. For example, if atom 1 contributes 8 contractions and atom 2 contributes 8 contractions, the `coord_types` list should be of length 16, where the first 8 elements correspond to contractions for atom 1 and the last 8 elements correspond to contractions for atom 2. These possibilities are illustrated in the code cell below using the Kr<sub>2</sub> molecule as an example:

In [42]:
# load STO-6G basis set information
basis_dict = parse_nwchem("data_sto6g.nwchem")

# create basis for Kr2 using cartesian coordinates for all generalized contractions
cartesian_basis = make_contractions(basis_dict, 
                                    ["Kr", "Kr"], 
                                    np.array([[0, 0, 0], [1, 0, 0]]),
                                    coord_types="cartesian")
print([shell.coord_type for shell in cartesian_basis])
print()

# create basis for Kr2 using spherical coordinates for all generalized contractions
spherical_basis = make_contractions(basis_dict, 
                                    ["Kr", "Kr"], 
                                    np.array([[0, 0, 0], [1, 0, 0]]),
                                    coord_types="spherical")
print([shell.coord_type for shell in spherical_basis])
print()

# create basis for Kr2 using spherical coordinates for all generalized contractions
# except the last one, which is specified to be cartesian
mixed_basis = make_contractions(basis_dict, 
                                ["Kr", "Kr"], 
                                np.array([[0, 0, 0], [1, 0, 0]]),
                                coord_types=["spherical"] * 15 + ["cartesian"])
print([shell.coord_type for shell in mixed_basis])

TypeError: make_contractions() got an unexpected keyword argument 'coord_types'

## 1.4 Linear Transformations of Basis Functions

In `gbasis`, the user can linearly transform the basis functions before computing the desired properties.
All of the higher level functions have the keyword argument `transform` to specify the matrix that
transforms the basis set. The transformation is applied to the left, i.e.,

$$\begin{equation}
 \psi_i = \sum_j T_{ij} \phi_j
\end{equation}$$

where {$\phi_{j}$} is the basis set before transformation and {$\psi_{i}$} is the basis function after transformation.
The number of basis functions depends on the coordinate systems specified for each shell. 

Examples involving such transformations are provided in the tutorial notebooks on evaluations and integrals wherever molecular rather than atomic orbitals are involved in calculations.